In [2]:
import numpy as np
from typing import Callable


class AlphaBeta(object):
    def __init__(self,
                 board: np.ndarray,
                 depth: int,
                 current_player: int,
                 enemy_player: int,
                 state_eval_fn: Callable[[np.ndarray], float]):
        self.board = board
        self.depth = depth
        self.__alpha = (-1) * np.inf
        self.__beta = np.inf
        self.current_player = current_player
        self.enemy_player = enemy_player
        self.evaluate = state_eval_fn

    def update_state(self, state: np.ndarray) -> None:
        self.state = state

    def predict_state(self):
        def AlphaBetaMax(state, alpha, beta, depth_left):
            if depth_left == 0:
                return (state, self.evaluate(state))
            best_state = None
            for possible_state in possible_states(state, self.current_player):
                found_state, score = AlphaBetaMin(
                    possible_state,
                    alpha,
                    beta,
                    depth_left - 1)
                if score >= beta:
                    return (found_state, beta)
                if score > alpha:
                    alpha = score
                    best_state = found_state
            return (best_state, alpha)

        def AlphaBetaMin(state, alpha, beta, depth_left):
            if depth_left == 0:
                return (state, (-1) * self.evaluate(state))
            best_state = None
            for possible_state in possible_states(state, self.enemy_player):
                found_state, score = AlphaBetaMin(
                    possible_state,
                    alpha,
                    beta,
                    depth_left - 1)
                if score <= alpha:
                    return (found_state, alpha)
                if score < beta:
                    beta = score
                    best_state = found_state
            return (best_state, beta)

        return AlphaBetaMax(self.board.copy(), self.__alpha, self.__beta, self.depth)


def MeasureOneToTwoFactory(first_player: int, second_player: int):
    def MeasureOneToTwo(state: np.ndarray) -> float:
        first_player_moves = count_possible_states(state, first_player)
        second_player_moves = count_possible_states(state, second_player)
        return first_player_moves - (2 * second_player_moves)
    return MeasureOneToTwo


def count_possible_states(state: np.ndarray, player: int) -> int:
    first_move = player_neighbourhood(state, player).sum() - player
    second_move = (state == 1).sum() - 1
    return first_move * second_move


def possible_states(state: np.ndarray, current_player: int):
    possible_moves = find_possible_player_moves(state, current_player)
    for move_vector in possible_moves:
        partial_state = move_player(state, current_player, move_vector)
        possible_removals = find_possible_removals(partial_state)
        for remove_pos in possible_removals:
            yield remove_square(partial_state, remove_pos)


def find_possible_player_moves(state: np.ndarray, current_player: int) -> list:
    pos_x, pos_y = np.where(state == current_player)
    pos_x = pos_x[0]
    pos_y = pos_y[0]
    player_neighbourhood = state[pos_x - 1: pos_x + 2, pos_y - 1: pos_y + 2]
    x_positions = (np.where(player_neighbourhood == 1)[0] + pos_x - 1)
    y_positions = (np.where(player_neighbourhood == 1)[1] + pos_y - 1)
    return np.array((x_positions, y_positions)).T.tolist()


def player_neighbourhood(state: np.ndarray, player: int) -> np.ndarray:
    pos_x, pos_y = np.where(state == player)
    pos_x = pos_x[0]
    pos_y = pos_y[0]
    return state[pos_x - 1: pos_x + 2, pos_y - 1: pos_y + 2]


def find_possible_removals(state: np.ndarray) -> list:
    return np.array(np.where(state == 1)).T.tolist()


def move_player(state: np.ndarray, current_player: int, move_vector: tuple) -> np.ndarray:
    result = state.copy()
    result[move_vector[0], move_vector[1]] = current_player
    result[move_vector[0], move_vector[1]] = 1
    return result


def remove_square(state: np.ndarray, square_pos: tuple) -> np.ndarray:
    result = state.copy()
    result[square_pos[0], square_pos[1]] = 0
    return result


In [3]:
def print_state(state: np.ndarray):
    print(state[1:-1, 1:-1])

In [9]:
def print_state(state: np.ndarray):
    print(state[1:-1, 1:-1])

__state = np.zeros((9,9))
state = np.ones((7,7))
__state[1:8, 1:8] = 1
state = __state

player_position = (1,4)
enemy_position = (7,4)
state[player_position] = 2
state[enemy_position] = 3
alpha_beta = AlphaBeta(state, 1, 2, 3, MeasureOneToTwoFactory(2,3))
alpha_beta.predict_state()

NoneType

In [12]:
MeasureOneToTwoFactory(2,3)(state)

-230.0

In [6]:
possible_states(state, 3)

<generator object possible_states at 0x7f7470e028d0>

In [167]:
def possible_states(state: np.ndarray, current_player: int):
    possible_moves = find_possible_player_moves(state, current_player)
    for move_vector in possible_moves:
        partial_state = move_player(state, current_player, move_vector)
        possible_removals = find_possible_removals(partial_state)
        for remove_pos in possible_removals:
            yield remove_square(partial_state, remove_pos)

def find_possible_player_moves(state: np.ndarray, current_player: int) -> list:
        pos_x, pos_y = np.where(state == current_player)
        pos_x = pos_x[0]
        pos_y = pos_y[0]
        player_neighbourhood = state[pos_x - 1: pos_x + 2, pos_y - 1: pos_y + 2]
        x_positions = (np.where(player_neighbourhood == 1)[0] + pos_x - 1)
        y_positions = (np.where(player_neighbourhood == 1)[1] + pos_y - 1)
        return np.array((x_positions, y_positions)).T.tolist()

def find_possible_removals(state: np.ndarray) -> list:
    return np.array(np.where(state == 1)).T.tolist()
    
def move_player(state: np.ndarray, current_player: int, move_vector: tuple) -> np.ndarray:
    result = state.copy()
    result[move_vector[0], move_vector[1]] = current_player
    result[move_vector[0], move_vector[1]] = 1
    return result

def remove_square(state: np.ndarray, square_pos: tuple) -> np.ndarray:
    result = state.copy()
    result[square_pos[0], square_pos[1]] = 0
    return result

def simple_state_eval(state: np.ndarray, current_player: int) -> float:
    pass

In [135]:
np.where(player_neighbourhood)[1] + pos_y - 1

array([3, 4, 5, 3, 4, 5])

In [96]:
x_positions = (np.where(player_neighbourhood == 1)[0] + pos_x - 1)
y_positions = (np.where(player_neighbourhood == 1)[1] + pos_y - 1)
possible_moves = np.array((x_positions, y_positions)).T
np.array((x_positions, y_positions)).T.tolist()

[[1, 3], [1, 5], [2, 3], [2, 4], [2, 5]]

In [102]:
possible_removals = np.array(np.where(state == 1)).T.tolist()

In [108]:
for move_x, move_y in possible_moves:
    partial_state = state.copy()
    partial_state[move_x, move_y] = 2
    partial_state[pos_x, pos_y] = 1
    possible_removals = np.array(np.where(partial_state == 1)).T.tolist()
    try:
        possible_removals.remove([move_x, move_y])
    except ValueError:
        pass
    for remove_x, remove_y in possible_removals:
        possible_state = partial_state.copy()
        possible_state[remove_x, remove_y] == 0

In [81]:
state[[1, 1, 2, 2, 2], [3, 5, 3, 4, 5]]

array([1., 1., 1., 1., 1.])

In [80]:
state

array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 2., 1., 1., 1., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 1., 1., 1., 3., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])